## Import Important Libaries

In [1]:
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import numpy as np
import time 
import os

import geopy
from geopy.geocoders import Nominatim

%matplotlib inline

## Dataframe detail informations 

In [2]:
"""
Just uncomment and run the program the code to view the details required
"""

df = pd.read_csv('data_100000.csv')

# df.head()           # view the dataframe
# df.info()           # view the columns data types and row nan details
# df.describe()       # view the dataframe mean, count, std, and other statics
# df.dtypes           # view the columns data types
# df.columns          # view the name of every avaliable dataframe column names.

In [3]:
print(df.latitude.isnull().sum())
print(df.longitude.isnull().sum())
print(df.zip_code.isnull().sum())

8035
8035
35034


## Address to Longtitude and Latitude

In [4]:
def lat_long(address):
    
    # GeoPy to get longtitude and latitude 
    geolocator = Nominatim(user_agent="Address_GeoLocator")
    location = geolocator.geocode(address)
    lat_long = [location.latitude, location.longitude]
    return lat_long

## Longtitude and Latitude to Address

In [5]:
def reverse_address(latitude, longitude):
    
    # GeoPy to get addresses 
    geolocator = Nominatim(user_agent="Geo_AddressLocator")
    geo_code = geolocator.reverse(f"{str(latitude)}, {str(longitude)}")
    adrs = geo_code.raw
    adrs = adrs['address']
    return adrs

In [6]:
df[df.on_street_name.notnull()].head(1)

,crash_date,crash_time,borough,zip_code,latitude,longitude,location,on_street_name,off_street_name,cross_street_name,...,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code1,vehicle_type_code2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5
1,2017-05-06T00:00:00.000,13:00,BRONX,10472.0,40.829052,-73.85038,"(40.829052, -73.85038)",CASTLE HILL AVENUE,BLACKROCK AVENUE,NaN,...,NaN,NaN,NaN,NaN,3665311,Sedan,NaN,NaN,NaN,NaN


## Fill null value of 
---
* zip_code
* borough
* on_street_name

using "latitude" and "longitude"

In [ ]:
geo_list = df[(df.latitude.notnull()) & (df.on_street_name.isnull())].index.tolist()

per_loop_data = 50
total_loop = int((len(geo_list)/per_loop_data)+1)

error_limit = 0  

print("Total null values", len(geo_list))

while True:
    for attempt in range(total_loop):
        
        try:
            geo_list = df[(df.latitude.notnull()) & (df.longitude.notnull()) & (df.on_street_name.isnull())].index.tolist()
            for ind in geo_list[:per_loop_data]:
                lat = df.latitude[df.index == ind].values[0]
                lon = df.longitude[df.index == ind].values[0]

                adrs = reverse_address(lat, lon)

                try:
                    zip_code = adrs['postcode']
                except:
                    zip_code = None
                
                try:
                    bor = adrs["suburb"]
                except:
                    zip_code = None
                
                try:
                    street = adrs['road']
                except:
                    zip_code = None

                df.loc[ind,"zip_code"] = zip_code
                df.loc[ind,"borough"] = bor
                df.loc[ind,"on_street_name"] = street
                error_limit = 0
        
        except Exception as e: 
            error_limit += 1
            if error_limit > 5:
                break
            
            print(e)
            print("Error!! Re-attempt")
            print(len(geo_list))
            
            time.sleep(3)
            
            
        if len(geo_list) == 0 or error_limit > 5:
            break
    
print("End of the process", len(geo_list))

Total null values 25179


In [31]:
txt = "banan erer ther"

x = txt.replace("banan", "apples")
y = txt.replace("the", "apples")

print(x)
print(y)

apples erer ther
banan erer applesr


## Fill null value of 
---
* latitude
* longitude

using "zip_code" and "borough"

In [ ]:
zip_list = df[(df.latitude.isnull()) & (df.zip_code.notnull())].index.tolist()
per_loop_data = 25
total_loop = int((len(zip_list)/per_loop_data)+1)

error_limit = 0   # if limi the loop to analysis the error

print("Total null values", len(zip_list))

while True:
    for attempt in range(total_loop):
        
        try:
            zip_list = df[(df.latitude.isnull()) & (df.zip_code.notnull())].index.tolist()
            for ind in zip_list[:per_loop_data]:
                bor = df.borough[df.index == ind].values
                zip_code = df.zip_code[df.index == ind].values
                codi = lat_long(f"{zip_code} {bor} NYC")
                df.loc[ind,"latitude"] = codi[0]
                df.loc[ind,"longitude"] = codi[1]
                error_limit = 0
                
        except Exception as e: 
            error_limit += 1
            if error_limit > 5:
                break
                
            print(e)
            print("Error!! Re-attempt")
            print(len(zip_list))
            time.sleep(3)
            
    
    if len(zip_list) == 0 or error_limit > 5:
        break
    
print("End of the process", len(zip_list))

In [ ]:
# Filling null value of address consume lot s time and memory so save address into new dire to work fresh.

if not os.path.exists("data_files"):
    os.makedirs("data_files")
df.to_csv(r'./data_files/filling_missing_address.csv', index=False, compression=compression_opts)

In [ ]:
print(df.latitude.isnull().sum())
print(df.longitude.isnull().sum())

In [ ]:
df = pd.read_csv(r'./data_files/filling_missing_address.csv')

"""
combin the date and time and convert data
type to date type to short accident base 
on time period
"""
df[["period"]] = (df["crash_date"] + ' ' + df["crash_time"])  # combine date and time
df.period = df.period.astype('datetime64[ns]')                # convert column to datetime 


"""
convert the "crash_date" to "datetime" data 
type to futher break data and find futher 
feature like date, data, month
"""
df["crash_date"] = df["crash_date"].astype('datetime64[ns]')  # convert colum to date time format
df["year"] = df["crash_date"].dt.year                         # get the year
df["month"] = df["crash_date"].dt.month                       # get the month
df["day"] = df["crash_date"].dt.day                           # get the day
df["weekday"] = df["crash_date"].dt.weekday                   # figure out which day it was
df["hour"] = pd.to_datetime(df['crash_time'], format='%H:%M').dt.hour  # grabing just hour value

"""
Based on the week days find accident happen
on weekday or on weekend.
"""
df["day_status"] = 0                               # "weekday"
df["day_status"].loc[df['weekday'] >4] = 1         # "weekend"

"""
Based on the month information finding 
at what session more safty needs to be concern.
"""
df["month_status"] = 0                                                      # "fall"
df.loc[(df["month"] < 3) | (df["month"] >= 12), "month_status"] = 1         # "winter"
df.loc[(df["month"] < 6) & (df["month"] >= 3), "month_status"] = 2         # "spring"
df.loc[(df["month"] < 9) & (df["month"] >= 6), "month_status"] = 3         # "summer"

"""
Based on the hour and weekday or weekenday 
finding what high accident chance day and hour.
"""
df["hour_status"] = 0                                                                  # "Evening"
df.loc[(df["hour"] < 18) & (df["day_status"] == "weekday"), "hour_status"] = 1         # "working_hour"
df.loc[(df["hour"] < 7) & (df["day_status"] == "weekday"), "hour_status"] = 2          # "night"
df.loc[(df["hour"] < 16) & (df["day_status"] == "weekday"), "hour_status"] = 3         # "day"
df.loc[(df["hour"] < 7) & (df["day_status"] == "weekday"), "hour_status"] = 4          # "night"

In [ ]:
# finding total number of accident victim per row.
df["total_victim"] = df.number_of_persons_injured + df.number_of_persons_killed

##  Null values graphical presentatin and understanding

In [ ]:
print('Drop the column where missing value is above 10% else just remove row')
print('Data size:', df.shape[0])
print('Data limit: ',int(0.1* df.shape[0]))

sn.heatmap(df.isnull(), yticklabels= False, cbar= False, cmap= 'Blues');

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
df.isna().sum() # viwe rows with missing values

In [ ]:
# data on these columns are very incorrent mistpyes plus similar columns were missing lots of data
# df = df.drop(columns=['vehicle_type_code1','contributing_factor_vehicle_1'])

# Replace 'nan' to 'Unspecified'
df["contributing_factor_vehicle_1"] = df["contributing_factor_vehicle_1"].fillna('Unspecified')

# Replace 'nan' to 'Unknown'
df["vehicle_type_code1"] = df["vehicle_type_code1"].fillna('Unknown')

# create heave light small etc catagory . 2 wheller or not.
t = df["vehicle_type_code1"].unique().tolist()
t.sort()
t

In [ ]:
for i in df.columns:
    print()
    print(f"********** {i} **********")
    print(df[f'{i}'].unique())
    print(df[f'{i}'].nunique())

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
# After attemping to get most of null values and extra values drop any columns that exceed the 10% null limit

drop_col = ["crash_date", "crash_time", "location"]

null_limit = 0.1
for i in df.columns:
    if df[f'{i}'].isna().sum() >=  int(null_limit * df.shape[0]):
        drop_col.append(i)
        
df = df.drop(columns=drop_col)

In [ ]:
# Cheeck if there are any dublicate rows
duplicate = df[df.duplicated()] 
print("Duplicate Rows :") 
duplicate

In [ ]:
df = df.dropna()  # or 'nan' is '0'

In [ ]:
df.columns

In [ ]:
df = df[['period','latitude', 'longitude', 'number_of_persons_injured',
       'number_of_persons_killed', 'number_of_pedestrians_injured',
       'number_of_pedestrians_killed', 'number_of_cyclist_injured',
       'number_of_cyclist_killed', 'number_of_motorist_injured',
       'number_of_motorist_killed', 'collision_id', 'year', 'month',
       'day', 'weekday', 'hour', 'day_status', 'month_status', 'hour_status',
       'total_victim']]

In [ ]:
df = df.sort_values(by=['period'])
df = df.reset_index()
df = df.drop(["index"], axis=1)

In [ ]:
df.head()

In [ ]:
column_headers = df.columns
dataset2 = df
feature_col = dataset2.shape[1]
plt_col = 2
plt_row = int((len(column_headers)/plt_col) + 1)


fig = plt.figure(figsize=(15, 25))
plt.suptitle('Subtitle for diagram', fontsize=20)
for i in range(feature_col):
    plt.subplot(plt_row, plt_col, i + 1)
    f = plt.gca()
    f.set_title(dataset2.columns.values[i])
    vals = np.size(dataset2.iloc[:, i].unique())
    
    # This help ploting process easier
    if vals >= 100:
        vals = 100
    
    plt.hist(dataset2.iloc[:, i], bins=vals, color='#3F5D7D')
# plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.set_tight_layout(True)


In [ ]:
## Correlation every feature
corr = df.corr()
sn.set(font_scale=2.8)
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize = (50,40))
sn.heatmap(corr, annot=True,mask=mask);

In [ ]:
## Correlation with independent variable (Note: Models like RF are not linear like these)

fig = "number_of_motorist_killed"
df.corrwith(df[f"{fig}"]).plot.bar(figsize = (20, 10), 
                                                        title = f"{fig}",
                                                        fontsize = 15,
                                                        rot = 45, grid = True);